# 단층 퍼셉트론으론 XOR 문제 못품

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
  print(device)
  torch.cuda.manual_seed_all(777)

cuda


In [3]:
X = torch.FloatTensor([[0,0],[0,1],[1,0],[1,1]]).to(device)
y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

In [4]:
linear = nn.Linear(2,1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear,sigmoid).to(device)

In [5]:
# criterion = F.binary_cross_entropy()
criterion = nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [6]:
for step in range(10001):
  hypothesis = model(X)
  cost = criterion(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if step % 1000 ==0:
    print(step, cost.item())

0 0.7273974418640137
1000 0.6931471824645996
2000 0.6931471824645996
3000 0.6931471824645996
4000 0.6931471824645996
5000 0.6931471824645996
6000 0.6931471824645996
7000 0.6931471824645996
8000 0.6931471824645996
9000 0.6931471824645996
10000 0.6931471824645996


In [7]:
#test시 with torch.no_grad() 사용하기 
with torch.no_grad():
  hypothesis = model(X)
  print(type(hypothesis))
  print(hypothesis)
  predicted = (hypothesis > 0.5).float()
  acc = (predicted== y).float().mean()
  # print(hypothesis)

<class 'torch.Tensor'>
tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], device='cuda:0')


# backpropagation by hands

In [17]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [30]:
w1 = torch.Tensor(2,2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2,1).to(device)
b2 = torch.Tensor(1).to(device)

In [31]:
w1, b1, w2,b2

(tensor([[-5.0814e+20,  3.0704e-41],
         [ 0.0000e+00,  0.0000e+00]], device='cuda:0'),
 tensor([-5.0814e+20,  3.0704e-41], device='cuda:0'),
 tensor([[-5.0814e+20],
         [ 3.0704e-41]], device='cuda:0'),
 tensor([-5.0814e+20], device='cuda:0'))

In [32]:
def sigmoid(x):
  return 1.0/(1.0+torch.exp(-x))

In [33]:
def sigmoid_prime(x):
  return sigmoid(x)*(1.0-sigmoid(x))

In [34]:
for step in range(10001):
  z1 = torch.add(torch.matmul(X,w1),b1)
  a1 = sigmoid(z1)
  z2 = torch.add(torch.matmul(a1,w2),b2)
  y_pred = sigmoid(z2)
  
  cost = -torch.mean(y*torch.log(y_pred)+(1.0-y)*torch.log(1.0-y_pred))

  d_y_pred = (y_pred-y)/(y_pred*(1.0-y_pred)+1e-7)

  d_z2 = d_y_pred * sigmoid_prime(z2)
  d_b2 = d_z2
  d_w2 = torch.matmul(torch.transpose(a1,0,1),d_b2)

  d_a1 = torch.matmul(d_b2,torch.transpose(w2,0,1)) 
  d_z1 = d_a1 * sigmoid_prime(z1)
  d_b1 = d_z1
  d_w1 = torch.matmul(torch.transpose(X,0,1),d_b1)

  w1 = w1 - 0.1*d_w1
  b1 = b1-0.1*torch.mean(d_b1,0)
  w2 = w2 - 0.1*d_w2
  b2 = b2-0.1*torch.mean(d_b2,0)
  if step%1000==0:
    print('-'*50)
    print(z1)
    print(w1,b1)
    print(step, cost.item())

--------------------------------------------------
tensor([[-5.0814e+20,  3.0704e-41],
        [-5.0814e+20,  3.0704e-41],
        [-1.0163e+21,  6.1408e-41],
        [-1.0163e+21,  6.1408e-41]], device='cuda:0')
tensor([[-5.0814e+20,  3.0704e-41],
        [ 0.0000e+00,  0.0000e+00]], device='cuda:0') tensor([-5.0814e+20,  3.0704e-41], device='cuda:0')
0 nan
--------------------------------------------------
tensor([[-5.0814e+20,  3.0704e-41],
        [-5.0814e+20,  3.0704e-41],
        [-1.0163e+21,  6.1408e-41],
        [-1.0163e+21,  6.1408e-41]], device='cuda:0')
tensor([[-5.0814e+20,  3.0704e-41],
        [ 0.0000e+00,  0.0000e+00]], device='cuda:0') tensor([-5.0814e+20,  3.0704e-41], device='cuda:0')
1000 nan
--------------------------------------------------
tensor([[-5.0814e+20,  3.0704e-41],
        [-5.0814e+20,  3.0704e-41],
        [-1.0163e+21,  6.1408e-41],
        [-1.0163e+21,  6.1408e-41]], device='cuda:0')
tensor([[-5.0814e+20,  3.0704e-41],
        [ 0.0000e+00,  0.0

# MLP로 해결

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device== 'cuda':
  torch.cuda.manual_seed_all(777)

In [36]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)

In [45]:
model = nn.Sequential(
    nn.Linear(2,10,bias=True),
    nn.Sigmoid(),

    nn.Linear(10,10, bias = True),
    nn.Sigmoid(),

    nn.Linear(10,10, bias = True),
    nn.Sigmoid(),

    nn.Linear(10,1, bias = True),
    nn.Sigmoid()
).to(device)

In [46]:
criterion = nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [48]:
for epoch in range(10001):
  hypothesis = model(X)

  cost = criterion(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 1000 == 0:
    print(epoch, cost.item())

0 0.00019202051043976098
1000 0.00015628107939846814
2000 0.00013155638589523733
3000 0.00011347897816449404
4000 9.967898949980736e-05
5000 8.881498069968075e-05
6000 8.00374400569126e-05
7000 7.283960439963266e-05
8000 6.680420483462512e-05
9000 6.167787068989128e-05
10000 5.723704816773534e-05


In [50]:
with torch.no_grad():
  hypothesis = model(X)
  predicted = (hypothesis > 0.5).float()
  acc = (predicted == y).float().mean()
  print(predicted)
  print(y)
  print(acc)

tensor([[0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')
tensor([[0.],
        [1.],
        [1.],
        [0.]], device='cuda:0')
tensor(1., device='cuda:0')
